In [1]:
%%time 

import numpy as np
import pandas as pd
from dateutil.relativedelta import *
from pandas.tseries.offsets import *

pd.set_option('display.max_columns', None)
comp = pd.read_csv('~/misp_data/comp_1959-2019.csv', index_col=0)
crsp = pd.read_csv('~/misp_data/crsp_1959-2019.csv', index_col=0)
beta = pd.read_csv('~/misp_data/capm_beta_weekly_1-3y_1959-2019.csv', index_col=0)

comp['jdate'] = pd.to_datetime(comp['jdate'])
comp.permno = comp.permno.astype(int)

beta['date']=pd.to_datetime(beta['DATE'].astype(str))
beta['jdate']=beta['date']+MonthEnd(0)
beta['permno'] = beta.index
beta = beta.drop_duplicates(subset=['permno', 'jdate'])

<string>:2: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
/software/python-anaconda-2020.02-el7-x86_64/lib/python3.7/site-packages/numpy/lib/arraysetops.py:569: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


CPU times: user 1min 3s, sys: 6.31 s, total: 1min 10s
Wall time: 1min 11s


In [2]:
FF = pd.read_csv('~/misp/lookup_tables/FF3.csv')
FF['date'] = FF['Unnamed: 0']
FF['date'] = pd.to_datetime(FF['date'].astype(str), format='%Y%m%d')
FF['jdate'] = FF['date']+MonthEnd(0)
FF = FF[['jdate', 'date', 'RF']]
FF['RF'] = (1+FF['RF']/100).pow(365)-1

beta = beta.merge(FF[['jdate', 'RF']],  on=['jdate'])
beta = beta.drop_duplicates(subset=['permno', 'jdate'])

In [3]:
def perform_valuation(df, val_method='dcf', **kwargs):
    if val_method == 'dcf':
        df['PV'] = dcf_valuation(df, **kwargs)
    else:
        raise ValueError(
            f'Invalid valuation method {val_method}.')
    return df


def dcf_valuation(df, n=10, rp=0.06, tg=0.06):
    beta = df['b_mkt']
    rf = df['RF']
    r = rf + beta * rp  # CAPM discount rate
    num_firms = df.shape[0]
    PV = np.zeros(num_firms)
    for i in range(n):
        ni_in_period_i = df[f'ni_{i+1}']
        PV += ni_in_period_i / (1 + r) ** i
        if i == n:
            PV += ni_in_period_i * (1 + tg) / (((1 + r) ** n) * (r - tg + 1e-10))
    return PV

In [14]:
comp.sort_values(['permno','year'], ascending = [True,False], inplace=True)
for i in range(11):
    comp[f'ni_{i}'] = comp.groupby('permno')['ni'].shift(i)
#     comp[f'ni_{i}'] = comp[f'ni_{i}'].fillna(0)

In [15]:
comp = comp.dropna(subset=['ni_5'])

In [32]:
%%time
valdf = comp.merge(beta, on=['jdate', 'permno'])
valdf = perform_valuation(valdf, n=3)
valdf = valdf[['jdate_comp', 'permno', 'gvkey', 'year', 'PV', 'be']]
valdf.year = valdf.year + 1
val_crsp_mdf = valdf.merge(crsp, on=['year', 'permno'])
val_crsp_mdf['vt'] = val_crsp_mdf['PV']*1000
val_crsp_mdf['be'] = val_crsp_mdf['be']*1000

CPU times: user 3.99 s, sys: 2.88 s, total: 6.87 s
Wall time: 6.87 s


In [33]:
val_df1 = val_crsp_mdf[['vt', 'permno', 'ticker', 'me', 'be', 'year', 'jdate_comp', 'jdate_crsp',
                        'logret_3', 'logret_6', 'logret_24', 'cumret_3', 'cumret_6', 'cumret_24',
                        'logret_12', 'logret_36', 'logret_60', 'cumret_12', 'cumret_36', 'cumret_60',
                         'termlogret', 'termcumret']]
val_df1.sort_values(['permno','jdate_crsp'], ascending = [True,False], inplace=True)

/software/python-anaconda-2020.02-el7-x86_64/lib/python3.7/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """


In [34]:
val_df2 = val_df1.copy()
for m in [3, 6, 12, 24, 36, 60]:
    val_df2[f'vt+{m}'] = val_df1.groupby('permno')['vt'].shift(m)
    val_df2[f'me+{m}'] = val_df1.groupby('permno')['me'].shift(m)
    val_df2[f'be+{m}'] = val_df1.groupby('permno')['be'].shift(m)

In [35]:
val_df3 = val_df2.copy()
for m in [3, 6, 12, 24, 36, 60]:
    val_df3['vt'] = np.log(val_df2.vt)
    val_df3[f'vt+{m}'] = np.log(val_df2[f'vt+{m}'].astype(float))
    val_df3['me'] = np.log(val_df2['me'].astype(float))
    val_df3[f'me+{m}'] = np.log(val_df2[f'me+{m}'].astype(float))
    val_df3[f'r_t+{m}'] = val_df3[f'logret_{m}'].fillna(val_df3['termlogret']+np.log(1-0.35))
    val_df3[f'r*_t+{m}'] = val_df3[f'vt+{m}'] -  val_df3[f'vt']
    val_df3[f'r_unchg+{m}'] = - (1-0.975) * (val_df3['me'] -  val_df3['vt'])
    val_df3[f'r_chg+{m}'] = 0.975 * ( (val_df3[f'me+{m}'] -  val_df3[f'vt+{m}']) - ((val_df3[f'me'] -  val_df3['vt'])) )
    

/software/python-anaconda-2020.02-el7-x86_64/lib/python3.7/site-packages/ipykernel_launcher.py:3: RuntimeWarning: invalid value encountered in log
  This is separate from the ipykernel package so we can avoid doing imports until
/software/python-anaconda-2020.02-el7-x86_64/lib/python3.7/site-packages/ipykernel_launcher.py:4: RuntimeWarning: invalid value encountered in log
  after removing the cwd from sys.path.


In [36]:
val_df3 = val_df3[val_df3.year<=2014]
val_df3 = val_df3.drop_duplicates(subset=['permno', 'jdate_crsp'])

In [37]:
val_df3['misp'] = val_df3['me'] - val_df3['vt']
val_df3 = val_df3.dropna(subset=['misp', 'r_t+60'])

In [38]:
val_df3[val_df3.ticker=='WMT']

,vt,permno,ticker,me,be,year,jdate_comp,jdate_crsp,logret_3,logret_6,logret_24,cumret_3,cumret_6,cumret_24,logret_12,logret_36,logret_60,cumret_12,cumret_36,cumret_60,termlogret,termcumret,vt+3,me+3,be+3,vt+6,me+6,be+6,vt+12,me+12,be+12,vt+24,me+24,be+24,vt+36,me+36,be+36,vt+60,me+60,be+60,r_t+3,r*_t+3,r_unchg+3,r_chg+3,r_t+6,r*_t+6,r_unchg+6,r_chg+6,r_t+12,r*_t+12,r_unchg+12,r_chg+12,r_t+24,r*_t+24,r_unchg+24,r_chg+24,r_t+36,r*_t+36,r_unchg+36,r_chg+36,r_t+60,r*_t+60,r_unchg+60,r_chg+60,misp
569724,17.664101,55976,WMT,19.136860,73471000.0,2011,2010-01-31,2011-12-31,0.009141,0.129667,0.367526,0.009182,0.138449,0.444158,0.225147,0.469836,0.307239,0.252507,0.599731,0.359666,8.068636,3191.743813,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.009141,NaN,-0.036819,NaN,0.129667,NaN,-0.036819,NaN,0.225147,NaN,-0.036819,NaN,0.367526,NaN,-0.036819,NaN,0.469836,NaN,-0.036819,NaN,0.307239,NaN,-0.036819,NaN,1.472759
569723,17.664101,55976,WMT,19.128751,73471000.0,2011,2010-01-31,2011-11-30,0.084720,0.050447,0.351221,0.088413,0.051741,0.420802,0.303672,0.369734,0.339115,0.354825,0.447349,0.403705,8.068636,3191.743813,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.084720,NaN,-0.036616,NaN,0.050447,NaN,-0.036616,NaN,0.303672,NaN,-0.036616,NaN,0.351221,NaN,-0.036616,NaN,0.369734,NaN,-0.036616,NaN,0.339115,NaN,-0.036616,NaN,1.464650
569722,17.664101,55976,WMT,19.091036,73471000.0,2011,2010-01-31,2011-10-31,0.147107,0.177392,0.403000,0.158478,0.194099,0.496307,0.376085,0.461161,0.457473,0.456570,0.585913,0.580077,8.068636,3191.743813,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.147107,NaN,-0.035673,NaN,0.177392,NaN,-0.035673,NaN,0.376085,NaN,-0.035673,NaN,0.403000,NaN,-0.035673,NaN,0.461161,NaN,-0.035673,NaN,0.457473,NaN,-0.035673,NaN,1.426936
569721,17.664101,55976,WMT,19.002228,73471000.0,2011,2010-01-31,2011-09-30,0.101971,0.111111,0.365110,0.107351,0.117519,0.440672,0.335139,0.423843,0.423448,0.398135,0.527822,0.527219,8.068636,3191.743813,17.664101,19.136860,73471000.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.101971,0.000000,-0.033453,0.131266,0.111111,NaN,-0.033453,NaN,0.335139,NaN,-0.033453,NaN,0.365110,NaN,-0.033453,NaN,0.423843,NaN,-0.033453,NaN,0.423448,NaN,-0.033453,NaN,1.338128
569720,17.664101,55976,WMT,19.034273,73471000.0,2011,2010-01-31,2011-08-31,0.080160,0.164881,0.440348,0.083461,0.179253,0.553248,0.370477,0.407650,0.453568,0.448425,0.503281,0.573918,8.068636,3191.743813,17.664101,19.128751,73471000.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.080160,0.000000,-0.034254,0.092116,0.164881,NaN,-0.034254,NaN,0.370477,NaN,-0.034254,NaN,0.440348,NaN,-0.034254,NaN,0.407650,NaN,-0.034254,NaN,0.453568,NaN,-0.034254,NaN,1.370172
569719,17.664101,55976,WMT,19.025208,73471000.0,2011,2010-01-31,2011-07-31,-0.016772,0.130335,0.386949,-0.016632,0.139209,0.472481,0.296980,0.419573,0.446129,0.345789,0.521312,0.562252,8.068636,3191.743813,17.664101,19.091036,73471000.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.016772,0.000000,-0.034028,0.064183,0.130335,NaN,-0.034028,NaN,0.296980,NaN,-0.034028,NaN,0.386949,NaN,-0.034028,NaN,0.419573,NaN,-0.034028,NaN,0.446129,NaN,-0.034028,NaN,1.361107
569718,17.664101,55976,WMT,19.033333,73471000.0,2011,2010-01-31,2011-06-30,-0.030616,0.071355,0.353241,-0.030152,0.073962,0.423675,0.201022,0.403571,0.376576,0.222651,0.497161,0.457287,8.068636,3191.743813,17.664101,19.002228,73471000.0,17.664101,19.136860,73471000.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.030616,0.000000,-0.034231,-0.030327,0.071355,0.000000,-0.034231,0.100939,0.201022,NaN,-0.034231,NaN,0.353241,NaN,-0.034231,NaN,0.403571,NaN,-0.034231,NaN,0.376576,NaN,-0.034231,NaN,1.369232
569717,17.664101,55976,WMT,19.075658,73471000.0,2011,2010-01-31,2011-05-31,-0.035576,0.044584,0.395685,-0.034951,0.045593,0.485401,0.095032,0.445863,0.323655,0.099694,0.561837,0.382170,8.068636,3191.743813,17.664101,19.03

In [39]:
%%time
val_df3.to_csv('~/misp_data/decomp_dcf3y_hist_1960_2016.csv')

CPU times: user 1min 8s, sys: 379 ms, total: 1min 8s
Wall time: 1min 9s
